# Exploring Ebay Car Sales Data (Consolidated Analysis)
## Introduction
This notebook contains a consolidated analysis of the eBay Kleinanzeigen dataset. It incorporates the best data cleaning, outlier handling, and analysis techniques identified from various previous versions.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chardet
%matplotlib inline

## 1. Data Loading and Initial Cleaning

In [2]:
# Detect encoding to prevent errors
with open('autos.csv', 'rb') as file:
    encoding_result = chardet.detect(file.read())
    print(encoding_result)
autos = pd.read_csv('autos.csv', encoding=encoding_result['encoding'])

In [3]:
# Clean column names to snake_case and more descriptive names
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price_usd', 'abtest', 'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km', 'registration_month', 'fuel_type', 'brand', 'unrepaired_damage', 'ad_created', 'nr_pictures', 'postal_code', 'last_seen']

In [4]:
# Clean and convert numeric columns
autos['price_usd'] = autos['price_usd'].str.replace('$', '').str.replace(',', '').astype(int)
autos['odometer_km'] = autos['odometer_km'].str.replace('km', '').str.replace(',', '').astype(int)

In [5]:
# Drop columns with low variance or relevance
autos = autos.drop(['seller', 'offer_type', 'nr_pictures', 'abtest'], axis=1)

## 2. Outlier Analysis and Removal

In [6]:
# Filter out unrealistic prices and registration years
# Prices between $850 and $350,001 are considered realistic
autos = autos[autos['price_usd'].between(850, 350001)]
# Registration year must be between 1927 and 2016 (the year the data was crawled)
autos = autos[autos['registration_year'].between(1927, 2016)]

## 3. Translating Categorical Data

In [7]:
# Translate German categorical values to English for better understanding
category_translator = {'bus':'bus', 'limousine':'sedan', 'kleinwagen':'compact', 'kombi':'station_wagon', 'coupe':'coupe', 'suv':'suv', 'cabrio':'convertible', 'andere':'other'}
autos['vehicle_type'] = autos['vehicle_type'].map(category_translator)

fuel_translator = {'benzin':'gasoline', 'diesel':'diesel', 'lpg':'lpg', 'cng':'cng', 'hybrid':'hybrid', 'elektro':'electric', 'andere':'other'}
autos['fuel_type'] = autos['fuel_type'].map(fuel_translator)

## 4. Aggregate Analysis: Price and Mileage by Brand

In [8]:
# Identify the top 10 brands by number of listings
top_brands = autos['brand'].value_counts().index[:10]

brands_agg = {}

for brand in top_brands:
    brand_subset = autos[autos['brand'] == brand]
    mean_price = brand_subset['price_usd'].mean()
    mean_mileage = brand_subset['odometer_km'].mean()
    brands_agg[brand] = {'mean_price_usd': round(mean_price, 2), 'mean_mileage_km': round(mean_mileage, 2)}

brand_analysis_df = pd.DataFrame.from_dict(brands_agg, orient='index')
print(brand_analysis_df.sort_values(by='mean_price_usd', ascending=False))

## 5. Visualization: Average Price vs. Average Mileage

In [9]:
fig, ax = plt.subplots(figsize=(12, 8))
brand_analysis_df.plot(kind='scatter', x='mean_mileage_km', y='mean_price_usd', ax=ax)

for idx, row in brand_analysis_df.iterrows():
    ax.annotate(idx, (row['mean_mileage_km'], row['mean_price_usd']), fontsize=12)

plt.title('Average Price vs. Average Mileage for Top 10 Brands')
plt.xlabel('Average Mileage (km)')
plt.ylabel('Average Price ($)')
plt.grid(True)
plt.show()